In [1]:
import pandas as pd
import scipy.stats as stats
import kagglehub
import matplotlib.pyplot as plt
import os
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as ols

/Users/siddharthgoradia/Documents/Loblaw-Data-Science/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# path = kagglehub.dataset_download("alexteboul/diabetes-health-indicators-dataset")

# print("Path to dataset files:", path)

Path to dataset files: /Users/siddharthgoradia/.cache/kagglehub/datasets/alexteboul/diabetes-health-indicators-dataset/versions/1


In [ ]:
# temp_data = pd.read_csv('/Users/siddharthgoradia/.cache/kagglehub/datasets/alexteboul/diabetes-health-indicators-dataset/versions/1/diabetes_012_health_indicators_BRFSS2015.csv')
# temp_data.rename(columns={"Diabetes_012": "Diabetes_binary"}, inplace=True)
# temp_data.to_csv('/Users/siddharthgoradia/.cache/kagglehub/datasets/alexteboul/diabetes-health-indicators-dataset/versions/1/diabetes_012_health_indicators_BRFSS2015.csv', index=False)

In [2]:
temp_data = pd.read_csv('../data/diabetes_012_health_indicators_BRFSS2015.csv')
temp_data.rename(columns={"Diabetes_012": "Diabetes_binary"}, inplace=True)
temp_data.to_csv('../data/diabetes_012_health_indicators_BRFSS2015.csv', index=False)
path = '../data'

In [3]:
csv_files = [f for f in os.listdir(path) if f.startswith("diabetes_") and f.endswith(".csv")]

df_list = [pd.read_csv(os.path.join(path, file)) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578052 entries, 0 to 578051
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_binary       578052 non-null  float64
 1   HighBP                578052 non-null  float64
 2   HighChol              578052 non-null  float64
 3   CholCheck             578052 non-null  float64
 4   BMI                   578052 non-null  float64
 5   Smoker                578052 non-null  float64
 6   Stroke                578052 non-null  float64
 7   HeartDiseaseorAttack  578052 non-null  float64
 8   PhysActivity          578052 non-null  float64
 9   Fruits                578052 non-null  float64
 10  Veggies               578052 non-null  float64
 11  HvyAlcoholConsump     578052 non-null  float64
 12  AnyHealthcare         578052 non-null  float64
 13  NoDocbcCost           578052 non-null  float64
 14  GenHlth               578052 non-null  float64
 15  

In [69]:
df['Diabetes_binary'] = df['Diabetes_binary'].apply(lambda x: 0 if x == 0 else 1)

## Appropriate statistical method of your choice, identify the risk factors associated with having diabetes based on the data.

In [70]:
categorical_vars = ['HighBP', 'HighChol', 'CholCheck', 'Smoker','Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies','HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'DiffWalk']
continuous_vars = ['BMI', 'MentHlth', 'PhysHlth', 'Age', 'Education','Income'] 

for var in categorical_vars:
    contingency_table = pd.crosstab(df[var], df['Diabetes_binary'])
    chi2, p, _, _ = stats.chi2_contingency(contingency_table)
    print(f'Chi-square test for {var}: p-value = {p}')

for var in continuous_vars:
    group0 = df[df['Diabetes_binary'] == 0][var]
    group1 = df[df['Diabetes_binary'] == 1][var]
    t_stat, p = stats.ttest_ind(group0, group1)
    print(f'T-test for {var}: p-value = {p}')

Chi-square test for HighBP: p-value = 0.0
Chi-square test for HighChol: p-value = 0.0
Chi-square test for CholCheck: p-value = 0.0
Chi-square test for Smoker: p-value = 0.0
Chi-square test for Stroke: p-value = 0.0
Chi-square test for HeartDiseaseorAttack: p-value = 0.0
Chi-square test for PhysActivity: p-value = 0.0
Chi-square test for Fruits: p-value = 3.6762259248600327e-265
Chi-square test for Veggies: p-value = 0.0
Chi-square test for HvyAlcoholConsump: p-value = 0.0
Chi-square test for AnyHealthcare: p-value = 2.1682047074019574e-39
Chi-square test for NoDocbcCost: p-value = 7.088444212367763e-178
Chi-square test for GenHlth: p-value = 0.0
Chi-square test for DiffWalk: p-value = 0.0
T-test for BMI: p-value = 0.0
T-test for MentHlth: p-value = 0.0
T-test for PhysHlth: p-value = 0.0
T-test for Age: p-value = 0.0
T-test for Education: p-value = 0.0
T-test for Income: p-value = 0.0


## Check whether individuals with a higher BMI are more likely to have diabetes (including both diabetes and pre-diabetes classifications)

In [71]:
bmi_no_diabetes = df[df['Diabetes_binary'] == 0]['BMI']
bmi_diabetes = df[df['Diabetes_binary'] == 1]['BMI']

t_stat, p_value = stats.ttest_ind(bmi_no_diabetes, bmi_diabetes, equal_var=False)

mean_diff = np.mean(bmi_diabetes) - np.mean(bmi_no_diabetes)
pooled_std = np.sqrt((np.var(bmi_diabetes, ddof=1) + np.var(bmi_no_diabetes, ddof=1)) / 2)
cohen_d = mean_diff / pooled_std

print(f'Independent t-test results:')
print(f'T-statistic = {t_stat:.5f}') 
print(f'P-value = {p_value:.5f}')
print(f'Cohen\'s d (effect size) = {cohen_d:.5f}')

Independent t-test results:
T-statistic = -172.15694
P-value = 0.00000
Cohen's d (effect size) = 0.60289


## Check whether individuals with a higher BMI are more likely to have diabetes but now after controlling for smoking status and physical activity (i.e., removing the effect of those variables).

In [73]:
if 'Smoker' not in df.columns or 'PhysActivity' not in df.columns:
    raise ValueError("The dataset must contain 'Smoker' and 'PhysActivity' columns")

anova_model = ols.ols('Diabetes_binary ~ BMI + Smoker + PhysActivity', data=df).fit()
anova_table = sm.stats.anova_lm(anova_model, typ=2)

stratified_results = []
for smoker_status in df['Smoker'].unique():
    for phys_activity_status in df['PhysActivity'].unique():
        subgroup = df[(df['Smoker'] == smoker_status) & (df['PhysActivity'] == phys_activity_status)]
        if len(subgroup) > 1:
            t_stat, p_value = stats.ttest_ind(subgroup[subgroup['Diabetes_binary'] == 0]['BMI'],
                                              subgroup[subgroup['Diabetes_binary'] == 1]['BMI'], equal_var=False)
            stratified_results.append({
                'Smoker': smoker_status,
                'PhysActivity': phys_activity_status,
                'T-Statistic': t_stat,
                'P-Value': p_value
            })

stratified_results_df = pd.DataFrame(stratified_results)

print("\nTwo-Way ANOVA Results (Controlling for Smoking and Physical Activity):")
print(anova_table)

print("\nStratified T-Test Results (BMI & Diabetes within Smoking/Physical Activity Groups):")
print(stratified_results_df)


Two-Way ANOVA Results (Controlling for Smoking and Physical Activity):
                    sum_sq        df             F  PR(>F)
BMI            4513.513940       1.0  31390.510744     0.0
Smoker          289.717427       1.0   2014.921885     0.0
PhysActivity    738.234077       1.0   5134.257926     0.0
Residual      83115.172211  578048.0           NaN     NaN

Stratified T-Test Results (BMI & Diabetes within Smoking/Physical Activity Groups):
   Smoker  PhysActivity  T-Statistic  P-Value
0     1.0           0.0   -67.101023      0.0
1     1.0           1.0   -90.793223      0.0
2     0.0           0.0   -56.874521      0.0
3     0.0           1.0  -102.822109      0.0
